In [1]:
from geo_tool import *
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import Markdown
%matplotlib widget
sns.set_theme()

In [2]:

from random import Random

def split_points(points, det, epsilon):
    l_zero = []
    e_zero = []
    g_zero = []

    for p in points:
        d = det(p, (-1.0, 0.0), (1.0, 0.1))
        if abs(d) < epsilon:
            e_zero.append(p)
        elif d > 0:
            g_zero.append(p)
        else:
            l_zero.append(p)        
    
    return l_zero, e_zero, g_zero

def viz(points, det, epsilon):
    l_zero, e_zero, g_zero = split_points(points, det, epsilon)

    print("len({ p == 0 }) = ", len(e_zero))
    print("len({ p < 0 }) = ", len(l_zero))
    print("len({ p > 0 }) = ", len(g_zero))
            
    scenes = Scene([PointsCollection(g_zero, color="red"),
                    PointsCollection(l_zero, color="blue"),
                    PointsCollection(e_zero, color="green"),],
                   [LinesCollection([[(-1.0 * 110, 0.0 ), (1.0 * 110, 0.1 * 110)]], color="magenta")])
    plot = Plot([scenes])
    plot.draw()

In [3]:
    
def det_lib_2x2(a, b, c):
    return np.linalg.det(np.array([[a[0] - c[0], a[1] - c[1]], [b[0] - c[0], b[1] - c[1]]]))

def det_my_2x2(a, b, c):
    m00 = a[0] - c[0]
    m10 = a[1] - c[1]
    m01 = b[0] - c[0]
    m11 = b[1] - c[1]
    return m00 * m11 - m10 * m01

def det_lib_3x3(a, b, c):
    return np.linalg.det(np.array([[a[0], a[1], 1], [b[0], b[1], 1], [c[0], c[1], 1]]))

def det_my_3x3(a, b, c):
    #  a[0]  a[1]   1
    #  b[0]  b[1]   1
    #  c[0]  c[1]   1
    return a[0] * b[1] + \
           a[1] * c[0] + \
           b[0] * c[1] - \
           b[1] * c[0] - \
           c[1] * a[0] - \
           b[0] * a[1]
dets = {
    "Biblioteczny 2x2": det_lib_2x2,
    "Własna implementacja 2x2": det_my_2x2,
    "Biblioteczna 3x3": det_lib_3x3,
    "Własna implementacja 3x3": det_my_3x3,
}

In [4]:
def compare_dets(points, epsilon):
    dets_results = {}

    for name, det in dets.items():
        l_zero, e_zero, g_zero = split_points(points, det, epsilon)
        dets_results[name] = (len(l_zero), len(e_zero), len(g_zero))       

    def cmp(name, det):
        results = [None] * len(dets)
        for i, kv in enumerate(dets.items()):
            name2 = kv[0]
            if name == name2:
                results[i] = "/"
                continue

            results[i] = (
                dets_results[name][0] - dets_results[name2][0],
                dets_results[name][1] - dets_results[name2][1],
                dets_results[name][2] - dets_results[name2][2],
            )
        return results
        
    dr = pd.DataFrame({ name: dets_results[name] for name, det in dets.items() }, index=["<0", "==0", ">0"])
    cmps = pd.DataFrame({ name: cmp(name, det) for name, det in dets.items() }, index=dets.keys())
    
    return dr, cmps

In [5]:
rnd = Random(42)
points = [(rnd.uniform(-1000, 1000), rnd.uniform(-1000, 1000)) for _ in range(10 ** 5)]
epsilon = 1e-2

for epsilon in [1e2, 1e1, 1e0, 1e-1, 1e-2, 1e-3]:
    display(Markdown(f"## ε = {epsilon}"))
    results, comparison = compare_dets(points, epsilon)
    display(Markdown("### Wyniki"))
    display(results)
    display(Markdown("### Porównanie"))
    display(comparison)

## ε = 100.0

### Wyniki

,Biblioteczny 2x2,Własna implementacja 2x2,Biblioteczna 3x3,Własna implementacja 3x3
<0,47294,47294,47294,47294
==0,4930,4930,4930,4930
>0,47776,47776,47776,47776


### Porównanie

,Biblioteczny 2x2,Własna implementacja 2x2,Biblioteczna 3x3,Własna implementacja 3x3
Biblioteczny 2x2,/,"(0, 0, 0)","(0, 0, 0)","(0, 0, 0)"
Własna implementacja 2x2,"(0, 0, 0)",/,"(0, 0, 0)","(0, 0, 0)"
Biblioteczna 3x3,"(0, 0, 0)","(0, 0, 0)",/,"(0, 0, 0)"
Własna implementacja 3x3,"(0, 0, 0)","(0, 0, 0)","(0, 0, 0)",/


...

## ε = 0.001

### Wyniki

,Biblioteczny 2x2,Własna implementacja 2x2,Biblioteczna 3x3,Własna implementacja 3x3
<0,49786,49786,49786,49786
==0,0,0,0,0
>0,50214,50214,50214,50214


### Porównanie

,Biblioteczny 2x2,Własna implementacja 2x2,Biblioteczna 3x3,Własna implementacja 3x3
Biblioteczny 2x2,/,"(0, 0, 0)","(0, 0, 0)","(0, 0, 0)"
Własna implementacja 2x2,"(0, 0, 0)",/,"(0, 0, 0)","(0, 0, 0)"
Biblioteczna 3x3,"(0, 0, 0)","(0, 0, 0)",/,"(0, 0, 0)"
Własna implementacja 3x3,"(0, 0, 0)","(0, 0, 0)","(0, 0, 0)",/


In [6]:
df = pd.DataFrame(points, columns=["x", "y"])

df["det_my_2x2"] = det_my_2x2((df.x, df.y), (-1.0, 0.0), (1.0, 0.1))
df["det_lib_2x2"] = df[["x", "y"]].apply(det_lib_2x2, args=((-1.0, 0.0), (1.0, 0.1)), axis=1)
df["det_my_3x3"] = det_my_3x3((df.x, df.y), (-1.0, 0.0), (1.0, 0.1))
df["det_lib_3x3"] =  df[["x", "y"]].apply(det_lib_3x3, args=((-1.0, 0.0), (1.0, 0.1)), axis=1)

df["diff"] = np.abs(df["det_my_2x2"] - df["det_my_3x3"])

df[df["det_my_2x2"] != df["det_my_3x3"]]

,x,y,det_my_2x2,det_lib_2x2,det_my_3x3,det_lib_3x3,diff
5,-562.724050,10.710576,77.593557,77.593557,77.593557,77.593557,1.421085e-14
7,299.768876,89.882961,149.689035,149.689035,149.689035,149.689035,2.842171e-14
12,914.426144,-326.810910,-745.164434,-745.164434,-745.164434,-745.164434,1.136868e-13
15,614.256547,459.463573,857.401492,857.401492,857.401492,857.401492,1.136868e-13
21,-544.203449,-421.224073,-788.127801,-788.127801,-788.127801,-788.127801,1.136868e-13
...,...,...,...,...,...,...,...
99971,458.746251,-853.744071,-1753.462766,-1753.462766,-1753.462766,-1753.462766,2.273737e-13
99977,-438.609164,-723.733294,-1403.705671,-1403.705671,-1403.705671,-1403.705671,2.273737e-13
99992,830.288978,30.439412,-22.250073,-22.250073,-22.250073,-22.250073,7.105427e-15
99997,-335.846120,-458.501354,-883.518096,-883.518096,-883.518096,-883.518096,1.136868e-13


In [18]:
%matplotlib widget
cols = ["det_my_2x2", "det_lib_2x2", "det_my_3x3", "det_lib_3x3"]
diff = pd.DataFrame(0.0, index=cols, columns=list(reversed(cols)))
for col1 in diff.index:
    for col2 in diff.columns:
        diff[col1][col2] = (df[col1] - df[col2]).abs().mean()

# scale = -np.ceil(np.log10(diff[diff != 0].min().min())) + 1
# diff *= 10 ** scale

sns.heatmap(diff, annot=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [17]:
%matplotlib widget
df.plot.scatter(x="x", 
                y="y", 
                c=-df["diff"], 
                colormap='viridis')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='x', ylabel='y'>

In [15]:
%matplotlib widget
df.plot.scatter(x="x", 
                y="y", 
                c=df["det_my_2x2"], 
                colormap='viridis')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='x', ylabel='y'>

In [48]:
viz(points, det_my_2x2, 1e1)

len({ p == 0 }) =  527
len({ p < 0 }) =  49536
len({ p > 0 }) =  49937


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
rnd = Random(42)
points = [(rnd.uniform(-1000, 1000), rnd.uniform(-1000, 1000)) for _ in range(10 ** 5)]

def det(a, b, c):
    return np.linalg.det(np.array([[a[0], a[1], 1], [b[0], b[1], 1], [c[0], c[1], 1]]))

viz(points, det_my_2x2, 1e2)

len({ p == 0 }) =  4930
len({ p < 0 }) =  47294
len({ p > 0 }) =  47776


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
rnd = Random(42)
points = [(rnd.uniform(-1000, 1000), rnd.uniform(-1000, 1000)) for _ in range(10 ** 5)]


viz(points, det_lib_2x2, 1e-1)

len({ p == 0 }) =  7
len({ p < 0 }) =  49781
len({ p > 0 }) =  50212


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
rnd = Random(42)
points = [(rnd.uniform(-10 ** 14, 10 ** 14), rnd.uniform(-10 ** 14, 10 ** 14)) for _ in range(10 ** 5)]


# własne 2x2
# biblioteczne 2x2
# własne 3x3
# biblioteczne 3x3

viz(points, det, 10e12)

len({ p == 0 }) =  4929
len({ p < 0 }) =  47291
len({ p > 0 }) =  47780


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
rnd = Random(42)
points = []
for _ in range(1000):
#     r = rnd.uniform(0, 100)
    r = 100.0
    a = rnd.uniform(0, 2 * np.pi)
    points.append((r * np.sin(a), r * np.cos(a)))

def det(a, b, c):
    return np.linalg.det(np.array([[a[0], a[1], 1], [b[0], b[1], 1], [c[0], c[1], 1]]))

viz(points, det, 1e1)

len({ p == 0 }) =  35
len({ p < 0 }) =  487
len({ p > 0 }) =  478


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …